# Model Evaluation

In [3]:
!ls /tmp/LSTM_input-dim=13_num-classes=13_hidden-dims=128_num-layers=2_bidirectional=True_use-layernorm=True_dropout=0.5713020228087161_learning-rate=0.009880117756170353_weight-decay=5.256755602421856e-07

In [62]:
import os
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
from examples.train import metrics

In [63]:
!ls /data2/breizhcrops/dgx

InceptionTime  LSTM	 OmniScaleCNN  TempCNN
L1C	       MSResNet  StarRNN       TransformerEncoder


In [71]:
logdir = "/data2/breizhcrops/dgx"
runs = ["LSTM","OmniScaleCNN","MSResNet","StarRNN","TempCNN","TransformerEncoder", "InceptionTime"]

def load(run):
    y_pred = np.load(os.path.join(logdir,run,"y_pred.npy"))
    y_true = np.load(os.path.join(logdir,run,"y_true.npy"))
    y_score = np.load(os.path.join(logdir,run,"y_score.npy"))
    field_ids = np.load(os.path.join(logdir,run,"field_ids.npy"))
    result = pd.DataFrame([y_pred,y_true,field_ids],index=["y_pred","y_true","field_ids"]).T.set_index("field_ids")
    return result, y_score

stats = list()
for run in runs:
    result, _ = load(run)
    stat = metrics(result.y_true,result.y_pred)
    stat["model"] = run
    stats.append(stat)
stats = pd.DataFrame(stats).set_index("model")

In [96]:
df = stats.T
df["RF"] = ""

models = ["RF","OmniScaleCNN", "TempCNN","MSResNet", "InceptionTime", "LSTM","StarRNN","TransformerEncoder"]
table = df[models].loc[["accuracy","recall_macro","f1_macro","f1_weighted","kappa"]]
table.index = ["overall accuracy","average accuracy","class-mean f-score","weighted f-score","kappa-metric"]


In [101]:
print(table.to_latex(float_format="%.4f"))

\begin{tabular}{llrrrrrrr}
\toprule
model & RF &  OmniScaleCNN &  TempCNN &  MSResNet &  InceptionTime &   LSTM &  StarRNN &  TransformerEncoder \\
\midrule
overall accuracy   &    &        0.7347 &   0.7948 &    0.7668 &         0.7710 & 0.8010 &   0.7893 &              0.8046 \\
average accuracy   &    &        0.5194 &   0.5541 &    0.5361 &         0.5338 & 0.5663 &   0.5552 &              0.5848 \\
class-mean f-score &    &        0.5138 &   0.5610 &    0.5491 &         0.5363 & 0.5766 &   0.5619 &              0.5909 \\
weighted f-score   &    &        0.7163 &   0.7874 &    0.7694 &         0.7669 & 0.7992 &   0.7891 &              0.8034 \\
kappa-metric       &    &        0.6498 &   0.7306 &    0.6972 &         0.7038 & 0.7396 &   0.7272 &              0.7453 \\
\bottomrule
\end{tabular}



In [78]:
df[models]OmniscaleCNNTempCNNMS-ResNetInceptionTimeLSTMStarRNNTransformer

model,OmniScaleCNN,MSResNet,TempCNN,LSTM,StarRNN,TransformerEncoder
accuracy,0.734671,0.766837,0.794830,0.800983,0.789313,0.804552
f1_macro,0.513839,0.549065,0.561041,0.576579,0.561890,0.590879
f1_micro,0.734671,0.766837,0.794830,0.800983,0.789313,0.804552
f1_weighted,0.716275,0.769380,0.787383,0.799199,0.789067,0.803417
kappa,0.649812,0.697171,0.730600,0.739637,0.727189,0.745349
precision_macro,0.570504,0.636418,0.617525,0.638349,0.647902,0.603700
precision_micro,0.734671,0.766837,0.794830,0.800983,0.789313,0.804552
precision_weighted,0.774037,0.783465,0.792034,0.801159,0.799870,0.802677
recall_macro,0.519379,0.536133,0.554077,0.566342,0.555245,0.584775
recall_micro,0.734671,0.766837,0.794830,0.800983,0.789313,0.804552


In [ ]:
OmniscaleCNNTempCNNMS-ResNetInceptionTimeLSTMStarRNNTransformer